In [34]:
import os
import pandas as pd
import numpy as np
import re
import string
import json
# -*- coding:utf-8 -*-

# 合并不同物种

In [ ]:
add_list = [i for i in os.listdir("output/文献标注-校对") if "_add" in i] # add为校对好的文本
col_dic =  {"样本量":"num","患者性别":"gender","年龄":"age","证据等级":"evi"}

for col_name, col_ab in col_dic.items():
    
    ori_l = [i for i in os.listdir("output/文献标注/") if col_ab in i]
    df_l = []
    
    for add in add_list:
        
        # 读取校对后的文档
        sped = add.replace("_add.xls","")
        df = pd.read_excel("output/文献标注-校对/%s" % add, header=0, index_col=0)
        df["ID"] = df.index
        # 读取含有支持证据的文档
        file_names = [i for i in ori_l if sped in i]
        dfs = pd.read_excel("output/文献标注/%s" %file_names[0], header=0, index_col=0)
        
        # 将支持证据添加至校对后的文档
        df[['%s文本' % col_name, 'old_start', 'old_end', '所在句子', 'new_start','new_end']] = dfs[['answer', 'old_start', 'old_end', 'sup_evi', 'new_start','new_end']]
        
        # 为添加原文
        for idx in df.index:
            with open("../2.Label_prepare/output/paper_ld/%s/%s.txt" % (sped, idx), "r") as f:
                txt = f.read()
            df.loc[idx, "Full"] = txt 
        df_l.append(df)

    dfm = pd.concat(df_l,axis=0)
    dfm.index = [dfm["物种名称"].iloc[i].replace(" ", "_").lower() + "_" + str(dfm.index[i]) for i in range(dfm.shape[0])]
    dfm.to_excel("output/文献合并/human_label_%s.xlsx" % col_ab)

# 合并不同标注

In [56]:
col_dic =  {"样本量":"num","患者性别":"gender","年龄":"age","证据等级":"evi"}
dfs_l = []
for col_name, col_ab in col_dic.items():
    dfm = pd.read_excel("output/文献合并/human_label_%s.xlsx" % col_ab, index_col=0)
    if col_ab == "evi":
        dfs_l.append(dfm[['%s文本' % col_name,"检测方式文本"]])
    else:
        dfs_l.append(dfm[['%s文本' % col_name]])
dfs = pd.concat(dfs_l,axis=1)

dfh = dfm[['物种名称', '是否是致病菌', '样本量', '患者性别', '年龄', '免疫状态','预后', '证据等级','检测方式', 'id','ID', "Full"]]
dfh = pd.concat([dfh,dfs],axis=1)
dfh = dfh.fillna("none")

dfh.loc[dfh['是否是致病菌']== "是", "是否是致病菌"] = "yes"
dfh.loc[dfh['是否是致病菌'] != "yes", "是否是致病菌"] = "no"

dfh = dfh.copy()
dfh['本文中是肺部病原的概率'] = 0
dfh.loc[dfh["是否是致病菌"]=="yes",'本文中是肺部病原的概率'] = 1
dfh['标注方式'] = '人工标注'
dfh['原文链接'] = dfh['id']

In [57]:
# 数据清洗
dfh.loc[dfh["样本量"]==1, "样本量"] =  "1"
dfh.loc[dfh["样本量"]=="1-10", "样本量"] =  "2-10"
dfh.loc[dfh["样本量"]=="10-100", "样本量"] =  "11-100"
dfh.loc[dfh["样本量"]=="10-101", "样本量"] =  "11-100"
dfh.loc[dfh["样本量"]=="10-102", "样本量"] =  "11-100"
dfh.loc[dfh["样本量"]=="＞100", "样本量"] =  ">100"

dfh.loc[dfh["年龄"]=="1-17", "年龄"] =  "1-17岁"
dfh.loc[dfh["年龄"]=="17-65", "年龄"] =  "18-65岁"
dfh.loc[dfh["年龄"]=="65以上", "年龄"] =  ">65岁"

dfh.loc[dfh["患者性别"]=="群体", "患者性别"] =  "none"

dfh.loc[dfh["免疫状态"]=="损伤", "免疫状态"] =  "抑制"
dfh.loc[dfh["免疫状态"]=="受损", "免疫状态"] =  "抑制"
dfh.loc[dfh["免疫状态"]=="正常", "免疫状态"] =  "none"

dfh.loc[dfh["预后"]=="无法确定", "预后"] = "none"
dfh.loc[dfh["预后"]=="治愈", "预后"] =  "none"

dfh.loc[dfh["证据等级"]=="8：文献报道", "证据等级"] =  "none"
dfh.loc[dfh["证据等级"]=="7：免疫学证据", "证据等级"] =  "3：血及无菌体液（如脓胸液）"
dfh.loc[dfh["证据等级"]=="5-2：气管吸出物或痰的分子诊断（PCR、基因组测序等）病原", "证据等级"] =  "3：血及无菌体液（如脓胸液）"
dfh.loc[dfh["证据等级"]=="4：气管镜活检组织培养出病原体,或者支气管肺泡灌洗液（BALF）定量培养>10**4", "证据等级"] =  "4：气管镜活检组织培养出病原体，或者支气管肺泡灌洗液（BALF）定量培养>10**4"

In [58]:
# 保存
dfh[['物种名称', 'ID', '本文中是肺部病原的概率', '是否是致病菌', '样本量', '样本量文本','年龄', '年龄文本', '患者性别', '患者性别文本', '免疫状态', '预后','证据等级', '证据等级文本','检测方式', '检测方式文本', 'Full', '原文链接', '标注方式']].to_csv("output/文献合并/human_label.csv")